In [1]:
import requests
import re
import time
import lxml.html
import openpyxl
import pandas as pd

from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
custom_header = {
    'referer' : 'http://www.riss.kr/index.do',
    #'referer' : 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=&onHanja=&strSort=&p_year1=&p_year2=&iStartCount=0&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }

# 검색키워드: 우울

In [3]:
#http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9A%B0%EC%9A%B8&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EC%9A%B0%EC%9A%B8

In [4]:
paper_link_list = []
#52
for i in range(52):
    before_url = "http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9A%B0%EC%9A%B8&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EC%9A%B0%EC%9A%B8"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

우울장애 환자에서 항우울제 치료가 삶의 질에 미치는 영향
우울취약성과 대인관계문제의 관계에서 능동적, 회피/분산적 정서조절양식의 매개효과
우울을 동반하거나 동반하지 않은 강박증상군과 우울증상군의 변별 : 자의식, 예방초점 및 자기비난을 중심으로
기혼여성의 우울유무에 따른 좋아하는 날씨(FKW) 그림의 반응 특성 차이연구
중년여성의 부부친밀감, 우울과 갱년기 증상의 관계
뇌졸중 후 우울변화가 인지 및 상지기능회복에 미치는 영향
시간사용중재가 뇌졸중 환자의 우울 및 재활동기에 미치는 영향
뇌졸중 환자의 수면의 질과 우울정도가 삶의 질에 미치는 영향
병원간호사의 자아탄력성과 사회적 지지가 우울감에 미치는 영향
연령, 성별, 안경원 경력에 따른 안경사 우울증 정도 분석
10
여고생의 신체만족도에 따른 자존감과 우울의 관계 및 인물화의 특성
우울과 공황증상을 가진 중년여성의 미술치료 단일사례 연구 : 부정적 정서표현과 자기인식을 중심으로
녹색활동이 노인의 스트레스 및 우울에 미치는 영향
원예치료가 시각장애 노인의 우울 및 자아존중감에 미치는 효과
지역사회 거주 노인의 영적안녕, 사회적 지지, 생활만족도 및 우울의 관계
집단미술치료가 청소년의 우울과 자살생각에 미치는 효과
외상 후 스트레스 장애 아동의 우울·불안 및 자기효능감 미술치료 단일사례연구
소규모 제조업 남성 근로자의 수면의 질, 직무 스트레스 및 피로와 우울과의 관련성
집단미술치료가 저소득층 여성의 우울과 스트레스 감소에 미치는 효과연구 : 천연비누공예 프로그램을 중심으로
학령기 아동의 우울, 인지왜곡, 기능적 학교생활에 관한 연구
20
만성통증 환자의 수용과 마음챙김이 재앙적 사고와 우울 및 통증에 미치는 영향 : 수용과 마음챙김의 조절효과
만성 폐쇄성 폐질환 환자의 증상경험, 자기효능감, 우울과 약물복용 이행 간의 관계
집단작업치료가 지역사회 거주 경증치매환자의 인지기능 및 우울에 미치는 영향
군인들의 우울 및 자살사고에 따른 풍경구성법(LMT) 반응특성연구
장루미보유 직장암 환자의 불안과 우울 및 관련요인

무용동작(舞踊動作) 치료(治療)가 폐경기 여성(女性)의 부안(不安), 우울(憂鬱) 및 삶의 질(質)에 미치는 효과(效果)
보조생식술 받은 여성의 난임 관련 삶의 질에 미치는 영향 요인: 우울과 극복력 중심으로
일 보건진료소 관할 지역 노인의 건강 관련 특성, 우울, 사회적 지지가 인지기능에 미치는 영향
커플관계자료 분석방법에 의한 기러기 엄마와 자녀의 자아존중감과 모-자녀 의사소통이 초기, 후기 청소년과 엄마의 우울에 미치는 영향
우울 지수에 따른 일과성 스트레스 및 향기요법 반응에 대한 HRV 분석
우울, 불안, 불면 환자에 대한 심리척도의 유용성과 상관성에 대한 임상연구
우울 경향과 腹募穴, 背兪穴 압통과의 관계 비교 연구
대학생의 우울과 취업스트레스가 진로성숙도에 미치는 영향
재가노인의 우울, 자아존중감, 일상생활수행능력 및 생활만족도
Beck 우울 척도 점수가 요통 환자의 한방 치료 초기에 미치는 영향에 대한 후향적 연구
230
한국인들의 항우울제에 대한 인식 및 태도에 관한 연구
봉독이 우울증 모델 흰쥐에게 미치는 영향
한국어판 우울증 선별도구(Patient Health Questionnaire-9, PHQ-9)의 표준화 연구
만성 조현병, 우울증, 조울병 환자의 자살사고 관련요인
취약계층 독거노인의 우울에 미치는 영향변인
농촌지역 주민의 우울 수준과 양생실천 정도
지역사회 재가노인의 우울과 건강행위와의 관계에서 분노표현 방식의 매개효과에 대한 성별차이
임신 중 우울에 영향을 미치는 심리사회적 요인
노인의 일상생활수행능력, 우울 및 주관적 건강상태와 영향요인: 사회경제적 상태와 가족지지를 중심으로
여성 독거노인의 우울증 경험 연구
240
뇌졸중 환자의 우울감 감소를 위한 스트레스 면역훈련을 기반으로 한 원예치료프로그램
중년여성의 성공적인 노화인식과 우울
다중회귀와 회귀나무를 활용한 군인 우울 예측 연구
연구논문 : 미용사의 직무스트레스와 우울증 및 여가활동의 관련성
농촌거주 여성의 변실금과 우울
만성적인 신체질환을 가진 환자의 우울 감소를 위한 전화기

중년기 성인의 죽음불안 예측 요인
성인 남성의 건강 관련 삶의 질에 영향을 미치는 요인에 관한 경로 분석
노인에서 우울증과 위험요인 및 인지기능과의 관계
농촌 여성노인의 죽음태도와 관련된 변인
부모와 자녀의 심리적 변인과 부모-자녀 의사소통이 중학생 남녀의 자살생각에 미치는 자기효과와 상대방효과 비교
한국 농촌에 거주하는 결혼 이주 여성의 심리적 특성에 관한 사례 분석
노인근로자의 직무특성 및 여가활동참여가 건강에 미치는 영향
알츠하이머병에서 아세틸콜린에스테라제 억제제에 대한 치료 반응과 관련된 요인
약물중독자를 위한 자기초월증진 프로그램의 효과: 혼합설계연구 적용
청소년 초기의 신체상 태도와 관련 변인들 간의 관계
430
우울증의 뇌파변화
뇌졸중 환자의 삶의 질 구조모형
항암화학요법을 받은 입원 암환자의 삶의 질에 영향을 미치는 요인
우울증을 동반한 알코올 의존 환자에서 rTMS 사용시 기분증상의 개선 정도에 대한 예비연구
여대생의 이상섭식행동에 미치는 영향요인
청소년의 휴대 전화 사용 실태 및 정신과적 문제와의 연관성
혈관성 우울증의 개념, 병인기전, 치료와 예후
말기암 환자 가족보호자의 삶의 질에 영향을 미치는 요인
지역사회 거주노인에서 주관적인 성공적 노화의 예측인자
성인남녀의 가족건강성 관련변인
440
60세 이상 노인 인구에서 우울증과 자살의 위험인자
낙상예방 프로그램이 재가노인의 체력과 심리기능에 미치는 효과
정신 약물과 탈모증
소아청소년 우울증에서의 환경적 위험 인자들과 임상적 의미
회전근 개 질환 환자에서 심리적 상태 및 삶의 질 평가
ICF 모델에 근거한 노인의 삶의 질 예측 모형
암 환자의 정신병리와 치료 목적에 대한 인지 및 삶의 질의 관계
발달협응장애 동반 유무에 따른 주의력결핍 과잉행동장애 아동들의 임상적 특성
의과 대학생의 신체부위 불만족과 성격특성에 대한 연구
중년 이후 여성에서 삶의 질 저하에 영향을 미치는 요인: 폐경 형태와 대사증후군 위험요인을 중심으로
450
류마티스 관절염 환자의 건강관련 삶의 질 영향요인 분석
증례 : 소

In [5]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7451eff5f3ac4b10ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=082bd24f49b40428ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=1aa94d3aba753ae5b36097776a77e665',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=83fb7a068de9d49effe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=2c8b1f7b37478d7cffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=c6fe7bf861414dfaffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=9d3defcdf0334e19e9810257f7042666',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e939ec9dffbe8d1db7998d826d417196',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=9eb22d49aa30b298ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [6]:
len(paper_link_list)

513

In [7]:
paper_url = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url.append(tmp_url)

100%|████████████████████████████████████████████████████████████████████████████████████████| 513/513 [00:00<?, ?it/s]


In [8]:
paper_url

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7451eff5f3ac4b10ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=082bd24f49b40428ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=1aa94d3aba753ae5b36097776a77e665',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=83fb7a068de9d49effe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=2c8b1f7b37478d7cffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=c6fe7bf861414dfaffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=9d3defcdf0334e19e9810257f7042666',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e939ec9dffbe8d1db7998d826d417196',
 'http://www.ris

In [9]:
len(paper_url)

513

In [10]:
write_wb = openpyxl.Workbook()
write_ws = write_wb.active

In [11]:
"""
write_ws = write_wb.active
write_ws['A1'] = 'title_kor'
write_ws['B1'] = 'title_eng'
write_ws['C1'] = 'author'
write_ws['D1'] = 'book'
write_ws['E1'] = 'year'
write_ws['F1'] = 'keyword'
write_ws['G1'] = 'abstract_kor'
write_ws['H1'] = 'abstract_eng'
"""

"\nwrite_ws = write_wb.active\nwrite_ws['A1'] = 'title_kor'\nwrite_ws['B1'] = 'title_eng'\nwrite_ws['C1'] = 'author'\nwrite_ws['D1'] = 'book'\nwrite_ws['E1'] = 'year'\nwrite_ws['F1'] = 'keyword'\nwrite_ws['G1'] = 'abstract_kor'\nwrite_ws['H1'] = 'abstract_eng'\n"

In [12]:
write_ws = write_wb.active
write_ws['A1'] = 'title'
write_ws['B1'] = 'author'
write_ws['C1'] = 'book'
write_ws['D1'] = 'year'
write_ws['E1'] = 'keyword'
write_ws['F1'] = 'abstract_kor'
write_ws['G1'] = 'abstract_eng'

In [13]:
start = time.time()

for i, url in enumerate(paper_url):
    res = requests.get(url, headers = custom_header)
    element = lxml.html.fromstring(res.text)

    try:
        tmp_title = element.cssselect('h3.title')
        tmp_title = tmp_title[0].text_content()
        tmp_title = tmp_title.split("=")
        title_kor = tmp_title[0].strip()
        #title_eng = tmp_title[1].strip()

        tmp_author = element.cssselect('div.infoDetailL > ul > li:nth-child(1) > div > p')
        author = tmp_author[0].text_content().strip()

        tmp_book = element.cssselect('div.infoDetailL > ul > li:nth-child(3) > div > p')
        book = tmp_book[0].text_content().strip()

        tmp_year = element.cssselect('div.infoDetailL > ul > li:nth-child(5) > div > p')
        year = tmp_year[0].text_content().strip()

        tmp_keyword = element.cssselect('div.infoDetailL > ul > li:nth-child(7) > div > p')
        keyword = tmp_keyword[0].text_content().strip()

        tmp_abstract = element.cssselect('div.text.off')
        abstract_kor = tmp_abstract[0].text_content().strip()
        abstract_eng = tmp_abstract[1].text_content().strip()

        print(i, 'paper clear')

    except:
        print(i, 'ERROR')
        continue


    write_ws.append([title_kor, author, book, year, keyword, abstract_kor, abstract_eng])
    write_wb.save('./2_우울.xlsx')
    
print("Running Time : ", time.time() - start)

0 paper clear
1 paper clear
2 paper clear
3 paper clear
4 paper clear
5 paper clear
6 paper clear
7 paper clear
8 paper clear
9 paper clear
10 paper clear
11 paper clear
12 paper clear
13 paper clear
14 paper clear
15 paper clear
16 paper clear
17 paper clear
18 paper clear
19 paper clear
20 paper clear
21 paper clear
22 paper clear
23 paper clear
24 paper clear
25 paper clear
26 paper clear
27 paper clear
28 paper clear
29 paper clear
30 paper clear
31 paper clear
32 paper clear
33 paper clear
34 paper clear
35 ERROR
36 paper clear
37 paper clear
38 paper clear
39 paper clear
40 paper clear
41 paper clear
42 paper clear
43 paper clear
44 paper clear
45 paper clear
46 paper clear
47 paper clear
48 paper clear
49 paper clear
50 paper clear
51 paper clear
52 paper clear
53 paper clear
54 paper clear
55 paper clear
56 paper clear
57 paper clear
58 paper clear
59 paper clear
60 paper clear
61 paper clear
62 paper clear
63 paper clear
64 paper clear
65 paper clear
66 paper clear
67 paper cl